In [12]:
import torch
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

#dataset_name = '01_RES_inference_SIM'
#dataset_name = '01_RES_inference_U373'

folder_path='/home/tony/data/gnntest/CTC' # TODO: update this path
#dataset_name='Fluo-C2DL-Huh7' # TODO: update this path
dataset_name='Fluo-N2DH-SIM+'
#dataset_name='PhC-C2DH-U373'
#dataset_name='Fluo-N2DL-HeLa'

# === Step 1: Load PyG graph data ===
pt_file = f"{folder_path}/{dataset_name}/01_RES_inference/pytorch_geometric_data.pt"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
data = torch.load(pt_file, map_location=device)

print("✅ Graph loaded.")
print(f"Node feature shape: {data.x.shape}")
print(f"Edge index shape: {data.edge_index.shape}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/tony/data/gnntest/CTC/Fluo-N2DH-SIM+/01_RES_inference/pytorch_geometric_data.pt'

In [ ]:
# === Step 2: Load edge weights ===

pt_weights_file = f"{folder_path}/{dataset_name}/01_RES_inference/raw_output.pt"
#pt_weights_file = f"pt-read/{dataset_name}/raw_output.pt"
edge_weights = torch.load(pt_weights_file, map_location='cpu').detach().cpu().numpy()

print("✅ Edge weights loaded.")
print(f"Edge weights shape: {edge_weights.shape}")


In [ ]:
# === Step 3: Build edge list from edge_index ===
edges = [(int(s), int(t)) for s, t in data.edge_index.t().tolist()]

# Optional: Save edge list
output_file = f"{folder_path}/{dataset_name}/01_RES_inference/edges.txt"

with open(output_file, 'w') as f:
    for src, tgt in edges:
        f.write(f"{src} {tgt}\n")

print(f"✅ Saved {len(edges)} edges to {output_file}")


In [ ]:
# === Step 4: Create NetworkX graph ===
G = nx.Graph()
G.add_edges_from(edges)

# === Step 5: Visualize with edge colors ===
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(8, 8))
nx.draw_networkx_nodes(G, pos, node_size=20, node_color='skyblue')
nx.draw_networkx_edges(
    G, pos,
    edgelist=edges,
    width=1.5,
    edge_color=edge_weights,
    edge_cmap=plt.cm.plasma,
)
plt.title("Graph with Edge Weights from raw_output.pt")
plt.axis('off')
plt.show()
